# Reto 4

In [767]:
#Importar librerías
import numpy as np
import pandas as pd
import math
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from random import randrange

In [768]:
#Leer el archivo con los datos
data_wine = pd.read_csv('./data/winequality-white.csv',sep=';')

In [769]:
#La variable dependiente es la última columna, las independientes son las anteriores
x= data_wine.iloc[:,0:11]
y= data_wine.iloc[:,11:12]

In [770]:
#Se divide el archivo para entrenamiento y test. Se reserven 10000 datos para test
xTrain, xTest, yTrain, yTest = train_test_split(x, y, test_size = 2000, random_state = 0)

In [771]:
#Se concatenan los datos de test
newData= pd.concat([xTrain,yTrain], axis = 1)

In [772]:
#De estos datos concatenados se escogen aleatoriamente 100,1000 y 2898 para diferentes modelos. 
#De aquí se vuelven a separar en x y y
dataTrain1= newData.sample(100)
xTrain1= dataTrain1.iloc[:,0:11]
yTrain1= dataTrain1.iloc[:,11]
dataTrain2= newData.sample(1000)
xTrain2= dataTrain2.iloc[:,0:11]
yTrain2= dataTrain2.iloc[:,11]
dataTrain3= newData.sample(2898)
xTrain3= dataTrain3.iloc[:,0:11]
yTrain3= dataTrain3.iloc[:,11]

In [773]:
#Función que calcula el error cuadrático medio a partir de ciertos datos y el vector de parámetros w
def calc_errors(x,y,w):
    err=0
    for i in range(0,x.shape[0]):
        err+=(y[i]-(x[i]*w).sum())**2
    return err

In [774]:
#Función que calcula el gradiente descendente, demora al correr
#numErrors es el número de errores consecutivos cuya media debe ser menor que el threshold 
#para que se asuma que el algoritmo terminó, este se escogió con ensayo y error.
#El threshold se define por la variable minError, este se escogió con ensayo y error.
def gradient_desc(nX,nY,numCiclos=29,minError=0.0001):
    a=nX.to_numpy()
    y=nY.to_numpy()
    #Se concatena un 1 al final de las variables para representar la constante w0
    x = np.ones((a.shape[0],a.shape[1]+1))
    x[:,:-1] = a
    wNumber= x.shape[1]
    numData= x.shape[0]
    #Se calcula la hessiana
    H=[[0] * wNumber]* wNumber
    for i in range(0, numData):
        xi= x[i]
        H+=xi.transpose()*xi
    ei=np.linalg.eigvals(H)
    #Se obtienen los valores propios y se calcula n como 20/lambdaMax, valor escogido con ensayo y error. 
    #Si se utiliza 2/lambdaMax que es lo que indica la teoría el algoritmo demora mucho en converger
    n=20/max(ei)
    ws= [0.01] * wNumber
    errorA=1
    errorP=0
    j=0
    #Variable que representa la media mínima del error cuadrático medio actual
    minT=10
    #El ciclo se ejecuta hasta que el error cuadrático medio sea similar al error cuadrático medio
    #de los numCiclos ciclos anteriores a este. Así se asume que el modelo convergió
    while minT > minError:
        #Se escoge una fila de datos aleatoriamente
        i= randrange(numData)
        #Se calcula ws con el algoritmo de descenso de gradiente estocástico
        g=(x[i]*ws).sum()
        e=(g-y[i])         
        ws= ws - n*e*x[i]
        j=(j+1)%numCiclos
        #Cada numCiclos ciclos se pregunta si el error se similar al error de hace numCiclos ciclos
        if j==0:
            errorP=errorA
            errorA=calc_errors(x,y,ws)
            #Se guarda solo el error mínimo, que es el que se usa para comparar
            if abs(errorP-errorA)< minT:
                minT=abs(errorP-errorA)
                #Se puede descomentar el siguiente print para observar como este valor disminuye
                #print(minT)
    return ws
    

In [775]:
#Cálculo del modelo 1
linearRegr1 = gradient_desc(xTrain1, yTrain1)
print(linearRegr1)

[ 0.00588704 -0.06517606  0.06955145  0.03747305  0.00585837  0.00411706
 -0.00164801  0.08088695  0.29858326  0.07213041  0.42434245  0.08163437]


In [776]:
#Cálculo del modelo 2
linearRegr2 = gradient_desc(xTrain2, yTrain2)
print(linearRegr2)

[ 0.15394957+0.j  0.01076167+0.j  0.01869771+0.j  0.0248744 +0.j
  0.01031289+0.j  0.01161147+0.j -0.00206134+0.j  0.0385042 +0.j
  0.10572849+0.j  0.02538108+0.j  0.39038961+0.j  0.03888514+0.j]


In [777]:
#Cálculo del modelo 3
linearRegr3 = gradient_desc(xTrain3, yTrain3)
print(linearRegr3)

[ 1.68711985e-01+0.j  1.41114305e-02+0.j  1.74897381e-02+0.j
  1.25598179e-02+0.j  1.05672644e-02+0.j  1.28648492e-02+0.j
 -2.33173380e-04+0.j  3.73870625e-02+0.j  1.01314644e-01+0.j
  2.35516821e-02+0.j  3.68180557e-01+0.j  3.77287131e-02+0.j]


In [778]:
#Función para calcular R^2 como 1-u/v, similar a como lo calcula sklearn para validar
def calc_r2(nX,nY,w):
    a=nX.to_numpy()
    y=nY.to_numpy()
    x = np.ones((a.shape[0],a.shape[1]+1))
    x[:,:-1] = a
    wNumber= x.shape[1]
    numData= x.shape[0]
    u=0
    v=0
    #ym es la media de todos los y
    ym= y.mean()
    err=0
    for i in range(0, numData):
        xi=x[i]
        yi=y[i]
        yp=(xi*w).sum()
        #u es la suma de cuadrados residuales (yi - yPredecido)
        u+=(yi-yp)**2
        #u es la suma total de cuadrados (yi - yMedia)
        v+=(yi-ym)**2
    r2=1-(u/v)
    return r2
    

In [779]:
#Cálculo de R^2 para el modelo 1
r21 = calc_r2(xTest, yTest,linearRegr1)
print(r21)

[0.19442683]


In [780]:
#Cálculo de R^2 para el modelo 2
r22 = calc_r2(xTest, yTest,linearRegr2)
print(r22)

[0.1523779+0.j]


In [781]:
#Cálculo de R^2 para el modelo 3
r23 = calc_r2(xTest, yTest,linearRegr3)
print(r23)

[0.11407457+0.j]


# Resultados
Se puede observar que el R^2 de todos los modelos es bastante bajo. Este valor es incluso más bajo que el valor de los modelos de sklearn. Al igual que en el reto 3 esto puede explicarse con el hecho que no se calculó el mínimo exacto, sino una aproximación utilizando descenso de gradiente. El modelo con menor R^2 es el tercero y el que tiene mayor R^2 es el primero, el cual por casualidad se ajustó mejor a los datos de test ya que no tenía, lo cual es probable y explica también por qué un modelo con más datos tiene peor criterio de clasificación que los demás.